# transforms

モデルが学習できるようにデータセットの処理を行う。  
また、学習精度向上のための処理もある。*5参照*  
```
from torchvision.transforms import ....
```

In [12]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

import matplotlib.pyplot as plt

## 先にデータを色々いじくってみる

In [13]:
d = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True
)

In [42]:
datasets.FashionMNIST

torchvision.datasets.mnist.FashionMNIST

In [16]:
d

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train

In [59]:
d.data[1]

tensor([[  0,   0,   0,   0,   0,   1,   0,   0,   0,   0,  41, 188, 103,  54,
          48,  43,  87, 168, 133,  16,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   1,   0,   0,   0,  49, 136, 219, 216, 228, 236, 255,
         255, 255, 255, 217, 215, 254, 231, 160,  45,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,  14, 176, 222, 224, 212, 203, 198, 196, 200,
         215, 204, 202, 201, 201, 201, 209, 218, 224, 164,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 188, 219, 200, 198, 202, 198, 199, 199, 201,
         196, 198, 198, 200, 200, 200, 200, 201, 200, 225,  41,   0,   0,   0],
        [  0,   0,   0,   0,  51, 219, 199, 203, 203, 212, 238, 248, 250, 245,
         249, 246, 247, 252, 248, 235, 207, 203, 203, 222, 140,   0,   0,   0],
        [  0,   0,   0,   0, 116, 226, 206, 204, 207, 204, 101,  75,  47,  73,
          48,  50,  45,  51,  63, 113, 222, 202, 206, 220, 224,   0,   0,   0],
        [  0,   0,   0,   0, 200, 222, 209, 20

In [33]:
d.root

'data'

In [35]:
d.train

True

In [36]:
len(d)

60000

In [19]:
type(d)

torchvision.datasets.mnist.FashionMNIST

In [14]:
d[0]

(<PIL.Image.Image image mode=L size=28x28>, 9)

In [21]:
img, label = d[0]
type(img), img.size, label

(PIL.Image.Image, (28, 28), 9)

tensorにしていない場合、PIL Imageという形式で保存されている。  
28かけ28のサイズで、ラベルが9なのが分かった。  
全ラベル数は分からない。

In [22]:
img.show()

In [23]:
d.targets.unique()

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [27]:
d.targets[100]

tensor(8)

In [56]:
for i in d.targets.unique():
    a = len(d.targets[d.targets == i])
    print(i)
    print(a)
print(len(d.targets))

tensor(0)
6000
tensor(1)
6000
tensor(2)
6000
tensor(3)
6000
tensor(4)
6000
tensor(5)
6000
tensor(6)
6000
tensor(7)
6000
tensor(8)
6000
tensor(9)
6000
60000


In [28]:
img.mode

'L'

L：白黒であり、1チャンネル  
RGB：カラー、3チャンネル

In [31]:
d.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [37]:
d.class_to_idx

{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}

**まとめ**
- d  
設定した概要を知れる
- data[i]  
指定した番号のデータの中身を知れる
- root  
どこにデータが入っているか。  
- len(d)  
データ数
- img, label = d[0]  
    - type(img)  
    0番目のデータの形状を知る  
    - img.size  
    サイズを知る  
    - label  
    ターゲットは何か  
    - img.show()  
    0番目の画像は何か  
- targets  
ラベルは何か
    - targets.unique()
    全種類を知れる  
    - len(d.targets[d.targets == i])  
    ラベルiが何個あるか確認  
- mode  
チャネルについて確認  
- d.classes  
画像の種類について確認  
- d.class_to_idx  
画像の種類と対応するラベルを指定  

## transforms

In [34]:
ds = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor(),
    target_transform = Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

モデルが計算できるようにone-hot変換してやる。  
例）ラベル3  
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]  

データの確認で、ラベルが10個あるのを知った。から、10を指定している。  
0を10個並べて(dim=0)(ゼロテンソル), 書込む場所を指定（index=torch.tensor(y)）, そこに1を書込む（value=1）

In [61]:
import torch.nn.functional as F

# num_classes でクラス数を指定
target_transform = Lambda(lambda y: F.one_hot(torch.tensor(y), num_classes=10))

別の書き方。  
ラムダ変換だから、lambda